In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the CSV files as numpy arrays
data1 = np.genfromtxt('clear_sorted_new_L4_match_miRNA.csv', delimiter=',')# shape (num_samples, input_dim1)
data2 = np.genfromtxt('clear_sorted_new_miRNA_match_L4.csv', delimiter=',')  # shape (num_samples, input_dim2)


# Normalize the data
#data1 = (data1 - np.mean(data1, axis=0)) / np.std(data1, axis=0)
#data2 = (data2 - np.mean(data2, axis=0)) / np.std(data2, axis=0)
#data3 = (data3 - np.mean(data3, axis=0)) / np.std(data3, axis=0)


2023-03-05 19:05:04.998133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 19:05:09.971827: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 19:05:09.971898: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 19:05:20.163185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
input_dim1 = data1.shape[1]
input_dim2 = data2.shape[1]
print("shapes:", input_dim1, input_dim2)

shapes: 258 2454


In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Define input layers for each dataset
input1 = layers.Input(shape=(input_dim1,))
input2 = layers.Input(shape=(input_dim2,))

# Define encoder layers for dataset 1
x1 = layers.Dense(512, activation="relu")(input1)
x1 = layers.Dense(256, activation="relu")(x1)
encoded1 = layers.Dense(64, activation="relu")(x1)

# Define encoder layers for dataset 2
x2 = layers.Dense(512, activation="relu")(input2)
x2 = layers.Dense(256, activation="relu")(x2)
encoded2 = layers.Dense(64, activation="relu")(x2)


# Concatenate the encoded representations of the three datasets
merged = layers.concatenate([encoded1, encoded2])

# Define decoder layers for the merged dataset
x4 = layers.Dense(128, activation="relu")(merged)
x4 = layers.Dense(256, activation="relu")(x4)
decoded = layers.Dense(input_dim1 + input_dim2, activation="sigmoid")(x4)

# Define the autoencoder model
model = keras.Model(inputs=[input1, input2], outputs=decoded)


encoder = keras.Model(inputs=[input1, input2], outputs=merged)

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model on the data
model.fit([data1, data2], np.concatenate([data1, data2], axis=1), epochs=2, batch_size=32)


Epoch 1/2
196/196 [==============================] - 13s 47ms/step - loss: 5.0165
Epoch 2/2
196/196 [==============================] - 9s 44ms/step - loss: 5.0020


In [5]:
# Define the encoder model
encoded_data = encoder.predict([data1, data2])
# Compile the encoder model
#encoder.compile(optimizer="adam", loss="mean_squared_error")
# Save the features to a CSV file
np.savetxt('latent_data.csv', encoded_data, delimiter=',')

196/196 [==============================] - 1s 4ms/step


In [6]:
encoded_data.shape

(6241, 128)